In [1]:
from flask import Flask, render_template, request, jsonify
from neo4j import GraphDatabase
import spacy
import pandas as pd
import joblib
import re
import os



app = Flask(__name__)
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")


!pip install transformers
!pip install sentence-transformers
!pip3 install truecase


## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
from sklearn.metrics.pairwise import cosine_similarity

## for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
import string

## for w2v
import gensim
import gensim.downloader as gensim_api

## for bert
import transformers

from sentence_transformers import SentenceTransformer


import spacy
from spacy import displacy

import truecase

!pip install py2neo
from py2neo import Graph

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\geeth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\geeth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\geeth\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\geeth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


#### Dictionary Cluster for various categories

In [2]:
dic_clusters = {}
dic_clusters["DISEASE"] = ['disease', 'condition', 'illness', 'condition', 'sickness']
dic_clusters["DRUGS"] = ['drug', 'medication', 'medicine', 'pills', 'remedy']
   
    
#Removing Punctuation, Null entries and stopwords

def Cleaning_1_1(dic_clusters):

  lst_stopwords = nltk.corpus.stopwords.words("english")

  for k,v in dic_clusters.items():
    for word in v:
      if len(word)<=2:
        v.remove(word)

    for i in lst_stopwords:
      if i in v:
        v.remove(i)

    new_list = v
    dic_clusters.update({k: list(filter(None, new_list))})

  return dic_clusters   

#Removing duplicates between the dictionaries

def Cleaning_1_2(dic_clusters):
  seen = set()
  repeated = set()
  new_list=[]

  for k,v in dic_clusters.items():
    for i in set(v):
      if i in seen:
        repeated.add(i)
      else:
        seen.add(i) 
    
  for i in repeated:
    x=0
    for k,v in dic_clusters.items():
      new_list2 = v
      if i in v:
        x=x+1
        if x>1:
          new_list2.remove(i)
     
      dic_clusters.update({k: new_list2})

Cleaning_1_1(dic_clusters)
Cleaning_1_2(dic_clusters)

#Cleaned Dictionary cluster

for k,v in dic_clusters.items():
    print(k, ": ", v[0:len(v)], "...", len(v),"\n")

DISEASE :  ['disease', 'condition', 'illness', 'condition', 'sickness'] ... 5 

DRUGS :  ['drug', 'medication', 'medicine', 'pills', 'remedy'] ... 5 



#### Question Preperation

In [3]:
#preprocessing and cleaning the question

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

#### Generates BERT embeddings for cleaned question

The 'utils_bert_embedding' function utilizes the BERT tokenizer and model to encode the input text, extract the BERT embeddings, and return them as a NumPy array. This function is helpful for obtaining contextualized word representations using the BERT model, which can be used for various downstream tasks such as text classification, named entity recognition, or question answering.


In [4]:
def utils_bert_embedding(txt, tokenizer, nlp):
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = nlp(idx)
    X = np.array(embedding[0][0][1:-1])
    return X


#### Cleaning Questions in Templates CSV

In [5]:
#preprocessing and cleaning method 2

'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text2(text, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

#### Linguistic Similarity

In [6]:
def Similarity_LinguisticApproach(question, templates):
  score = []
  for template in templates["Template_Clean3"]:
    sc=0
    for word in question.split():
      if word in template.split():
        sc=sc+1
    score.append(sc)

  return score    

#### Cypher Query Retrieval

In [7]:
def answer_Retrieval(Final_predictions, Question, templates,subject,selected_cat,qtype,predicate):
    print("cypher_query")
    for p in Final_predictions:
        #fetching the appropriate template
        template_index = int(p[1])
        #print(templates["Cypher_Clean"])
        #print(template_index)
        cypher_query = templates["Cypher_Clean"].iloc[template_index]
        query = cypher_query.replace('{first_item_name_}', subject)
        print(query)
        #print(selected_cat)
        #print(qtype)
        #print(predicate)
        if (qtype == 'true'):
            query = query.replace('{second_item_name_}', selected_cat)
            query = query.replace('{reltn_}', predicate)
        print(query)
    return query



### Query Generation

In [8]:
def query_generation(question,obj_text,selected_cat,qtype,predicate):
	print("Question ---",question)
	#Question = "Are there any medical conditions that often lead to hepatic failure"
	Question = question
	lst_stopwords = nltk.corpus.stopwords.words("english")
	#Question preparation
	Clean_q = utils_preprocess_text(Question, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords)
	print("Output of Question preparation ---",Clean_q)
 
          
          
	#Context Identification      
	tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
	nlp = transformers.TFBertModel.from_pretrained('bert-base-uncased')
	lst_mean_vecs = utils_bert_embedding(Clean_q, tokenizer, nlp).mean(0)  #generates BERT embeddings and calculate the mean vector of these embeddings
	X = np.array(lst_mean_vecs) #Convert the list of mean vectors (lst_mean_vecs) to a NumPy array (X).
    
	#dic_y --> keys = items of the dic_clusters,  --> values = mean vectors calculated using the utils_bert_embedding function 
	dic_y = {k:utils_bert_embedding(v, tokenizer, nlp).mean(0) for k,v
         in dic_clusters.items()}
	## compute cosine similarities
	similarities_q = np.array(
				[metrics.pairwise.cosine_similarity(X.reshape(1, -1), y.reshape(1, -1)).T.tolist()[0] 
				 for y in dic_y.values()]
				).T
	## adjust and rescale
	labels = list(dic_y.keys())
	print(similarities_q)
	print(labels)    
    
	#Even when there is no similarity, there is still a label assigned and the resulting probabilities sum up to 1 for each row.
	for i in range(len(similarities_q)):
		### assign randomly if there is no similarity
		if sum(similarities_q[i]) == 0:
		   similarities_q[i] = [0]*len(labels)
		   similarities_q[i][np.random.choice(range(len(labels)))] = 1
		### rescale so they sum = 1
		similarities_q[i] = similarities_q[i] / sum(similarities_q[i])
		print(similarities_q[i])    
	## classify the label with highest similarity score
	predicted_prob_q = similarities_q
	predicted_q = [labels[np.argmax(pred)] for pred in predicted_prob_q]
	print('Predicted Context :: ',predicted_q)

          
          
	# Template Matching
	templates = pd.read_csv(predicted_q[0] + ".csv", sep=",", engine="python")
	#Cleaning Templates with utils_preprocess_text2 
	lst_stopwords = nltk.corpus.stopwords.words("english")
	index=0
	for sentence in templates["Template"]:
		 templates.at[index,'Template_Clean3'] = utils_preprocess_text2(sentence, flg_lemm=True)
		 index=index+1
	#Cleaning Question with utils_preprocess_text2          
	Question_simil_clean= utils_preprocess_text2(Question, flg_lemm=True)
	model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2') #The model is responsible for generating sentence embeddings, which are vector representations of sentences that capture their semantic and contextual information.
	template_embeddings = model.encode(templates["Template_Clean3"])
	question_embeddings = model.encode(Question_simil_clean)
	#for every question we calculate how similar it is to each template          
	similarities2 = cosine_similarity(question_embeddings.reshape(1, -1), template_embeddings)
	for i in range(len(similarities2)):
		### rescale so they sum = 1
		similarities2[i] = similarities2[i] / sum(similarities2[i])
	## classify the label with highest similarity score
	predicted_prob = similarities2
	predicted = []
	second =[]

	for pred in predicted_prob:
	  x = np.argmax(pred)
	  predicted.append("T" + str(x))
	  #getting the second biggest similarity
	  temp = np.copy(pred)
	  temp[x] = -1
	  x2=np.argmax(temp)
	  second.append("T" + str(x2))
	index=0
	for sentence in templates["Template"]:
	  templates.at[index,'Template_Clean3'] = utils_preprocess_text2(sentence, flg_lemm=True, lst_stopwords=lst_stopwords)
	  index=index+1
	Question_ling_clean = utils_preprocess_text2(Question, flg_lemm=True, lst_stopwords=lst_stopwords)
	scores = []
	scores = Similarity_LinguisticApproach(Question_ling_clean, templates)
	count=0
	for i in scores:
	  count=count+i
	if count==0:
		scores.clear()
	predicted_prob = scores
	predicted2=[]
	if predicted_prob:
	  x = np.argmax(pred)
	  predicted2.append("T" + str(x)) 
	else:
	  predicted2.append(None)
	Final_predictions=[]

	i=0
	for p1, p2 in zip(predicted, predicted2):
		if p2:
			if p2 == p1 :
				final_Pred = p1
			elif p2==second[i]:
				final_Pred = second[i]
			else:
				final_Pred = p2
		else:
			final_Pred = p1 
		i=i+1
		Final_predictions.append(final_Pred)
	print("Final Predictions", Final_predictions)

          
          
          
          
          
	# Query Retrieval
	
	index=0
	for cyph_q in templates["Cypher"]:
	  new_cyph_q = re.sub("<+[\w]*>", "#", cyph_q)
	  templates.at[index,'Cypher_Clean'] = new_cyph_q
	  index=index+1
	  
	templates["Cypher_Clean"].head()
	print("\nFinal_predictions: ", Final_predictions)
	query = answer_Retrieval(Final_predictions, Question, templates,obj_text,selected_cat,qtype,predicate)
	print("Cypher Query: ",query)


	return query

In [ ]:

# Define the route for the home page
@app.route('/')
def home():
    return render_template('index.html')



# Define the route for the Cypher query - Student
@app.route('/convert_nlp_to_cypher', methods=['POST'])
def stu_query():
    # Get the natural language query from the HTML form
    query_text = request.form.get('nlp_query')
    obj_text = request.form.get('obj_text')
    qtype = request.form.get('qtype')
    selected_cat = request.form.get('selected_cat')
    predicate = request.form.get('predicate')
    cy_query = query_generation(query_text,obj_text,selected_cat,qtype,predicate)
    return cy_query



    

if __name__ == '__main__':
    app.run(port=8080)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2023 08:57:20] "GET / HTTP/1.1" 200 -


Question --- Which diseases are commonly associated with Anemia
Output of Question preparation --- disease commonly associated anemia


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

[[0.52727485 0.461806  ]]
['DISEASE', 'DRUGS']
[0.5330958 0.4669042]
Predicted Context ::  ['DISEASE']


...

Final Predictions ['T6']

Final_predictions:  ['T6']
cypher_query
MATCH (e:First_Item)-[r:Relation]->(c:Second_Item) WHERE e.Name =~ 'Anemia' AND NOT (e)-[:Relation {type: 'belongs to the drug family of'}]->() OPTIONAL MATCH (e)<-[r_inv:InverseRelation]-(c) RETURN e, r, c, r_inv
MATCH (e:First_Item)-[r:Relation]->(c:Second_Item) WHERE e.Name =~ 'Anemia' AND NOT (e)-[:Relation {type: 'belongs to the drug family of'}]->() OPTIONAL MATCH (e)<-[r_inv:InverseRelation]-(c) RETURN e, r, c, r_inv
Cypher Query:  MATCH (e:First_Item)-[r:Relation]->(c:Second_Item) WHERE e.Name =~ 'Anemia' AND NOT (e)-[:Relation {type: 'belongs to the drug family of'}]->() OPTIONAL MATCH (e)<-[r_inv:InverseRelation]-(c) RETURN e, r, c, r_inv


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

[[0.45459807 0.4224146 ]]
['DISEASE', 'DRUGS']
[0.51834835 0.48165165]
Predicted Context ::  ['DISEASE']


127.0.0.1 - - [24/Aug/2023 09:09:39] "POST /convert_nlp_to_cypher HTTP/1.1" 200 -


Final Predictions ['T6']

Final_predictions:  ['T6']
cypher_query
MATCH (e:First_Item)-[r:Relation]->(c:Second_Item) WHERE e.Name =~ 'Anemia' AND NOT (e)-[:Relation {type: 'belongs to the drug family of'}]->() OPTIONAL MATCH (e)<-[r_inv:InverseRelation]-(c) RETURN e, r, c, r_inv
MATCH (e:First_Item)-[r:Relation]->(c:Second_Item) WHERE e.Name =~ 'Anemia' AND NOT (e)-[:Relation {type: 'belongs to the drug family of'}]->() OPTIONAL MATCH (e)<-[r_inv:InverseRelation]-(c) RETURN e, r, c, r_inv
Cypher Query:  MATCH (e:First_Item)-[r:Relation]->(c:Second_Item) WHERE e.Name =~ 'Anemia' AND NOT (e)-[:Relation {type: 'belongs to the drug family of'}]->() OPTIONAL MATCH (e)<-[r_inv:InverseRelation]-(c) RETURN e, r, c, r_inv
